In [ ]:
import csv
import pandas as pd
import re
from tqdm import tqdm
import time

index_list=list(range(0,100000))
df = pd.read_csv('/content/drive/MyDrive/kmdb_csv.csv', encoding='euc-kr',index_col=False)

# 불필요한 열 삭제, 이름 수정, 결측치 처리
df=df.drop(['제작국가','제작년도','용도','출연','영화등록번호NO','키워드','영화등록번호ID','영문제명','원제명','유형','제작사','영화심의여부','대표영화심의일','대표영화심의번호','대표영화관람등급','대표개봉일','대표상영시간','최초등록일','최종수정일'],axis=1)
df.rename(columns={'각본':'출연'},inplace=True)
df.rename(columns={'줄거리':'키워드'},inplace=True)
df.rename(columns={'KMDBURL':'줄거리'},inplace=True)
df=df.fillna("")

# 불필요한 내용 제거 및 split
for a in ['장르','출연','키워드']:
    for i in range(100000):
        if len(df[a][i])>0:
            df[a][i]=df[a][i].split(',')
# 배우(역활) -> (역활) 제거
for i in range(100000):
    actors=[]
    for actor in df['출연'][i]:
        result = re.sub('\(.+?\)', '', actor)
        actors.append(result)
        df['출연'][i]=actors
# 영화 중 결측치가 4개 이상인 행 제거
for i in tqdm(range(100000)):
    count=0
    if len(df.loc[i]['장르'])<1:
        count=count+1
    if len(df.loc[i]['감독'])<1:
        count=count+1
    if len(df.loc[i]['출연'])<1:
        count=count+1
    if len(df.loc[i]['키워드'])<1:
        count=count+1
    if len(df.loc[i]['줄거리'])<1:
        count=count+1
    if count>1:
        df=df.drop([i])
df.reset_index(drop=True,inplace=True)

!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

In [ ]:
cd Mecab-ko-for-Google-Colab

In [ ]:
!bash install_mecab-ko_on_colab_light_220429.sh

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()
df['줄거리_pos_tagged'] = df['줄거리'].apply(lambda x: mecab.pos(x))

In [ ]:
b=[]
for ii in range(len(df3['줄거리_pos_tagged'])):
    a=[]
    for i in df['줄거리_pos_tagged'][ii]:
        if len(i[0])>1 and ('NNP' in i[1] or 'NNG' in i[1]):
            a.append(i[0])
    b.append(a)

In [ ]:
df['줄거리_fin']=b
df=df.drop('줄거리_pos_tagged',axis=1)

In [ ]:
for i in range(len(df['감독'])):
    df['감독'][i] = re.sub(r'\?', '', df['감독'][i])